Jegadeesh, N., & Titman, S. (1993). Returns to buying winners and selling losers: Implications for stock market efficiency. The Journal of finance, 48(1), 65-91.

reference:https://wrds-www.wharton.upenn.edu/pages/wrds-research/applications/python-replications/replicating-momentum-strategies-jegadeesh-and-titman-jf-1993-python/#momentum-strategies

In [2]:
# 读入数据
import janitor
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
from scipy import stats

datapath = "/Users/fuhk/Desktop/research/JT1995/input/"
msf = pd.read_sas(datapath+"msf.sas7bdat").clean_names()
msenames = pd.read_sas(datapath+"msenames.sas7bdat").clean_names()

/Users/anaconda3/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [57]:
# 做一张空表备用
# formation period
j_list = [3, 6, 9, 12]
# hold period
k_list = [3, 6, 9, 12]
columns = ['K=' + str(k) + '(E)' for k in k_list] + \
    ['K=' + str(k) + '(V)' for k in k_list]
indexs = list()
for j in j_list:
    for p in ["sell", "buy", "buy-sell"]:
        for q in ['mean', 't-stat']:
            indexs.append(f"J={j},{p},{q}")
# 创建一个DataFrame，列名是columns
table1 = pd.DataFrame(columns=columns, index=indexs)
table1

,K=3(E),K=6(E),K=9(E),K=12(E),K=3(V),K=6(V),K=9(V),K=12(V)
"J=3,sell,mean",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=3,sell,t-stat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=3,buy,mean",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=3,buy,t-stat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=3,buy-sell,mean",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=3,buy-sell,t-stat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=6,sell,mean",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=6,sell,t-stat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=6,buy,mean",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"J=6,buy,t-stat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# 样本选择
start_date = pd.to_datetime('1963-01-01')
end_date = pd.to_datetime('1989-12-31')

msf_use = msf[(msf['date'] >= start_date) & (msf['date'] <= end_date)][['permno','date','shrout','prc','ret']]
msenames_use = msenames[['permno','shrcd','exchcd','namedt','nameendt']]
msf_use = msf_use.conditional_join(
    msenames_use,
    ('permno', 'permno', '=='),
    ('date', 'namedt', '>='),
    ('date', 'nameendt', '<='),
    how = 'inner'
    )

msf_use.drop(columns = ('right','permno'), inplace = True)
msf_use.columns = msf_use.columns.droplevel(0)
msf_use = msf_use[(msf_use['exchcd'].isin([-2,-1,0,1,2])) & (msf_use['shrcd'].isin([10,11]))][['permno','date','shrout','prc','ret']] 
crsp_m = msf_use.copy()

In [114]:
# 做一张空表备用
# formation period
j_list = [3, 6, 9, 12]
# hold period
k_list = [3, 6, 9, 12]
columns = ['K=' + str(k) + '(E)' for k in k_list] + \
    ['K=' + str(k) + '(V)' for k in k_list]
indexs = list()
for j in j_list:
    for p in ["sell", "buy", "buy-sell"]:
        for q in ['mean', 't-stat']:
            indexs.append(f"J={j},{p},{q}")
# 创建一个DataFrame，列名是columns
table1 = pd.DataFrame(columns=columns, index=indexs)
table1

j_list = [3, 6, 9, 12]
k_list = [3, 6, 9, 12]
for J in j_list:
    for K in k_list:
        crsp_m['value'] = crsp_m['shrout'] * np.absolute(crsp_m['prc'])
        crsp_m['value'] = crsp_m.sort_values(by=['permno','date'],inplace = False).groupby('permno')['value'].shift(1)
        _tmp_crsp = crsp_m[['permno', 'date', 'ret']].sort_values(
            ['permno', 'date']).set_index('date')
        _tmp_crsp['ret'] = _tmp_crsp['ret'].fillna(0)
        _tmp_crsp['logret'] = np.log(1+_tmp_crsp['ret'])
        umd = _tmp_crsp.groupby(['permno'])['logret'].rolling(
            J, min_periods=J).sum()
        umd = umd.reset_index()
        umd['cumret'] = np.exp(umd['logret'])-1

        umd = umd.dropna(axis=0, subset=['cumret'])
        umd['momr'] = umd.groupby('date')['cumret'].transform(
            lambda x: pd.qcut(x, 10, labels=False))
        umd.momr = umd.momr.astype(int)
        umd['momr'] = umd['momr']+1

        umd['form_date'] = umd['date']
        umd['medate'] = umd['date']+MonthEnd(0)
        umd['hdate1'] = umd['medate']+MonthBegin(1)
        umd['hdate2'] = umd['medate']+MonthEnd(K)
        umd = umd[['permno', 'form_date', 'momr', 'hdate1', 'hdate2']]
        _tmp_ret = crsp_m[['permno', 'date', 'ret']]
        umd2 = crsp_m.conditional_join(
            umd,
            ("permno", "permno", "=="),
            ("date", "hdate1", ">="),
            ("date", "hdate2", "<="),
            how='inner',
            sort_by_appearance=True,
        )
        umd2.drop(columns=('right', 'permno'), inplace=True)
        umd2.columns = umd2.columns.droplevel(0)
        umd3 = umd2.groupby(['date', 'momr', 'form_date'])[
            'ret'].mean().reset_index()
        umd4 = umd2.groupby(['date', 'momr', 'form_date'])[['ret', 'value']].apply(lambda x: ((
            x['ret']*x['value']).sum()/x['value'].sum())).reset_index().rename(columns={0: 'vret'})
        umd3 = pd.merge(
            umd3, umd4, on=['date', 'momr', 'form_date'], how='inner')
        start_yr = umd3['date'].dt.year.min()+2
        umd3 = umd3[umd3['date'].dt.year >= start_yr]
        umd3 = umd3.sort_values(by=['date', 'momr'])
        # 计算均值和t-value
        ewret = umd3.groupby(['date', 'momr'])['ret'].mean().reset_index()
        ewstd = umd3.groupby(['date', 'momr'])['ret'].std().reset_index()
        ewret = ewret.rename(columns={'ret': 'ewret'})
        ewstd = ewstd.rename(columns={'ret': 'ewretstd'})
        ewretdat = pd.merge(ewret, ewstd, on=['date', 'momr'], how='inner')
        ewretdat = ewretdat.sort_values(by=['momr'])

        ewretdat.sort_values(by=['momr', 'date'], inplace=True)
        ewretdat2 = ewretdat.pivot(
            index='date', columns='momr', values='ewret')
        ewretdat2 = ewretdat2.add_prefix('port')
        ewretdat2 = ewretdat2.rename(
            columns={'port1': 'losers', 'port10': 'winners'})
        ewretdat2['long_short'] = ewretdat2['winners'] - ewretdat2['losers']

        mom_mean = ewretdat2[['winners', 'losers', 'long_short']].mean()
        table1.at[f'J={J}'+',sell' + ',mean',
                  f'K={K}(E)'] = mom_mean['losers'].round(4)
        table1.at[f'J={J}'+',buy' + ',mean',
                  f'K={K}(E)'] = mom_mean['winners'].round(4)
        table1.at[f'J={J}'+',buy-sell' + ',mean',
                  f'K={K}(E)'] = mom_mean['long_short'].round(4)

        table1.at[f'J={J}'+',sell' + ',t-stat',
                  f'K={K}(E)'] = '('+str(pd.Series(stats.ttest_1samp(ewretdat2['losers'], 0.0)).to_frame().T.iloc[0, 0].round(2))+')'
        table1.at[f'J={J}'+',buy' + ',t-stat',
                  f'K={K}(E)'] = '('+str(pd.Series(stats.ttest_1samp(ewretdat2['winners'], 0.0)).to_frame().T.iloc[0, 0].round(2))+')'
        table1.at[f'J={J}'+',buy-sell' + ',t-stat',
                  f'K={K}(E)'] = '('+str(pd.Series(stats.ttest_1samp(ewretdat2['long_short'], 0.0)).to_frame().T.iloc[0, 0].round(2))+')'

        # 计算加权均值和t-value
        ewret = umd3.groupby(['date', 'momr'])['vret'].mean().reset_index()
        ewstd = umd3.groupby(['date', 'momr'])['vret'].std().reset_index()
        ewret = ewret.rename(columns={'vret': 'vwret'})
        ewstd = ewstd.rename(columns={'vret': 'vwretstd'})
        ewretdat = pd.merge(ewret, ewstd, on=['date', 'momr'], how='inner')
        ewretdat = ewretdat.sort_values(by=['momr'])

        ewretdat.sort_values(by=['momr', 'date'], inplace=True)
        ewretdat2 = ewretdat.pivot(
            index='date', columns='momr', values='vwret')
        ewretdat2 = ewretdat2.add_prefix('port')
        ewretdat2 = ewretdat2.rename(
            columns={'port1': 'losers', 'port10': 'winners'})
        ewretdat2['long_short'] = ewretdat2['winners'] - ewretdat2['losers']

        mom_mean = ewretdat2[['winners', 'losers', 'long_short']].mean()
        table1.at[f'J={J}'+',sell' + ',mean',
                  f'K={K}(V)'] = mom_mean['losers'].round(4)
        table1.at[f'J={J}'+',buy' + ',mean',
                  f'K={K}(V)'] = mom_mean['winners'].round(4)
        table1.at[f'J={J}'+',buy-sell' + ',mean',
                  f'K={K}(V)'] = mom_mean['long_short'].round(4)

        table1.at[f'J={J}'+',sell' + ',t-stat',
                  f'K={K}(V)'] = '('+str(pd.Series(stats.ttest_1samp(ewretdat2['losers'], 0.0)).to_frame().T.iloc[0, 0].round(2))+')'
        table1.at[f'J={J}'+',buy' + ',t-stat',
                  f'K={K}(V)'] = '('+str(pd.Series(stats.ttest_1samp(ewretdat2['winners'], 0.0)).to_frame().T.iloc[0, 0].round(2))+')'
        table1.at[f'J={J}'+',buy-sell' + ',t-stat',
                  f'K={K}(V)'] = '('+str(pd.Series(stats.ttest_1samp(ewretdat2['long_short'], 0.0)).to_frame().T.iloc[0, 0].round(2))+')'
table1

,K=3(E),K=6(E),K=9(E),K=12(E),K=3(V),K=6(V),K=9(V),K=12(V)
"J=3,sell,mean",0.0115,0.0095,0.0095,0.009,0.0064,0.0057,0.0055,0.005
"J=3,sell,t-stat",(2.29),(1.96),(2.0),(1.93),(1.58),(1.49),(1.47),(1.37)
"J=3,buy,mean",0.0135,0.0145,0.0149,0.0153,0.0125,0.0124,0.0128,0.0131
"J=3,buy,t-stat",(3.44),(3.65),(3.72),(3.8),(3.63),(3.61),(3.71),(3.73)
"J=3,buy-sell,mean",0.002,0.005,0.0053,0.0064,0.0061,0.0067,0.0074,0.0081
"J=3,buy-sell,t-stat",(0.7),(2.03),(2.47),(3.4),(2.05),(2.71),(3.35),(4.04)
"J=6,sell,mean",0.0097,0.0085,0.0078,0.0085,0.0063,0.0049,0.0036,0.0044
"J=6,sell,t-stat",(1.87),(1.69),(1.59),(1.75),(1.5),(1.21),(0.93),(1.16)
"J=6,buy,mean",0.0164,0.0169,0.0172,0.0164,0.0139,0.0146,0.0151,0.0144
"J=6,buy,t-stat",(4.13),(4.2),(4.23),(4.06),(3.94),(4.07),(4.16),(3.97)
